Metra Bridge Data

URL - [https://services3.arcgis.com/6LvtIYUSMXW8Tb6o/ArcGIS/rest/services/Metra_Bridges/FeatureServer/0](https://services3.arcgis.com/6LvtIYUSMXW8Tb6o/ArcGIS/rest/services/Metra_Bridges/FeatureServer/0)

In [1]:
import restapi
import os
import time
from pathlib import Path

full_run_time_start = time.time()

url = "https://services3.arcgis.com/6LvtIYUSMXW8Tb6o/ArcGIS/rest/services"

# reference map service
ags = restapi.ArcServer(url)

ModuleNotFoundError: No module named 'restapi'

In [2]:
from pprint import pprint

for root, services in ags.walk():
    pprint("Folder: {}".format(root))
    print(f"Services: ")
    [print(service) for service in services]

NameError: name 'ags' is not defined

In [3]:
metra_lines = ags.getService("metra_lines_2018/FeatureServer")
lines = metra_lines.layer(0)
featureSet = lines.query(exceed_limit=True)

NameError: name 'ags' is not defined

In [4]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
from shapely.geometry import shape

df = gpd.read_file(str(featureSet.json))
df

NameError: name 'featureSet' is not defined

In [ ]:
import folium
import branca.colormap as cm

m = df.explore(
    column="LINE_NAME",
    style_kwds={"weight": 5},
    legend="LINE_NAME",
    legend_kwds={"caption": "Metra Lines"},
)

In [6]:
m

# Adding MBI Project Points

In [7]:
import folium
import zipfile
import io

from fastkml import kml

# read KML content
kmz = zipfile.ZipFile(
    "C:\\Users\\dane.parks\\Projects\\Chicago Metra\\MBI Rail Projects - Edited - 2.kmz",
    "r",
)  # extract zip file first, then read kmz file inside the extracted folder
kml_content = kmz.open("doc.kml", "r").read()  # kml content

# create KML object
k = kml.KML()
k.from_string(kml_content)

# read features from docs to folders to records and then extract geometries - in my case, Shapely points
docs = list(k.features())
folders = []
for d in docs:
    folders.extend(list(d.features()))

records = []

for f in folders:
    records.extend(list(f.features()))

geoms = [(element.name, element.geometry) for element in records]  # extract geometry

# Filtering MBI Project Points to Ones that Fall Into Metra Limits

In [8]:
def bounding_box(points):
    y_coordinates, x_coordinates = zip(*points)

    return [
        (
            (min(x_coordinates), min(y_coordinates)),
            (max(x_coordinates), max(y_coordinates)),
        )
    ]

In [9]:
points = []
i = 0

for feature in featureSet.json["features"]:
    for coordinates in feature["geometry"]["coordinates"]:
        if isinstance(coordinates[0], list):
            for lists in coordinates:
                points.append((lists[0], lists[1]))
                i += 1
        elif len(coordinates) != 2:
            for coordinate in coordinates:
                points.append((coordinate[0], coordinate[1]))
                i += 1
        else:
            points.append((coordinates[0], coordinates[1]))
            i += 1

In [10]:
metra_area = bounding_box(points)

# Setting the Map to the Bounding Area

In [11]:
m.fit_bounds(metra_area)
m

In [12]:
metra_area

[((41.4190191595046, -88.617388856149), (42.5857989558056, -87.5477796388403))]

In [13]:
filtered_points = []
for geom in geoms:
    if (geom[1].x > metra_area[0][0][1] and geom[1].x < metra_area[0][1][1]) and (
        geom[1].y > metra_area[0][0][0] and geom[1].y < metra_area[0][1][0]
    ):
        filtered_points.append(geom)

geoms = filtered_points

In [14]:
test = geoms[0]
test

('NS Br. B-512.06 Chicago, IL (Stoney Island)', Point(-87.585049, 41.720515))

In [15]:
for geom in geoms:
    folium.Marker(
        location=(geom[1].y, geom[1].x), fill_color="#43d9de", radius=2, popup=geom[0]
    ).add_to(m)

In [16]:
m

# Adding Station Markers

In [17]:
metra_lines = ags.getService("metra_stations_2018/FeatureServer")
lines = metra_lines.layer(0)
featureSet = lines.query(exceed_limit=True)

folium.GeoJson(
    featureSet.json,
    name="Metra Bridges",
    zoom_on_click=True,
    marker=folium.Circle(
        radius=6, fill_color="orange", fill_opacity=0.4, color="white", weight=1
    ),
    # tooltip=folium.GeoJsonTooltip(fields=["BRNO", "CROSSING", "LINE"]),
    # popup=folium.GeoJsonPopup(fields=["BRNO", "CROSSING", "FULLCODE", "CROSSING", "CITY", "LINE", "LINE_1", "MILE", "NOTES", "PLAN_17"]),
).add_to(m)

In [18]:
m

In [19]:
m.save("metra.html")

In [20]:
import fiona

fiona.supported_drivers["KML"] = "rw"

m.to_file("test.kml", driver="KML")

AttributeError: 'Map' object has no attribute 'to_file'

In [ ]:
folium.map.LayerControl("bottomright", collapsed=False).add_to(m)

In [ ]:
m